In [1]:
# Imaging plate with shifter -> Generates shifter csv file (imaging.csv 1 plate)
# Setup echo transfer -> Generates echo protocol csv (write_echo_csv())
    # Dropdown to select active library plate
    # Dropdown Select the crystal plate 
    # Specify how many wells in the crystal plate is to be used -> Deterimines how many library wells are used
# Select Batch number from dropdown. Click button to add timestamp to echo-transfer
# Create imaging.csv based on selected batch
# Harvesting step in shifter generates harvesting.csv and that gets consumed by script
# Generate LSDC puck excel file to be imported - sampleName in LSDC excel file would be "target-UID" target = Project.target

# Query by crystal ID get puck name, 
# Ligand identity from the library well
# Transfer volume and delta time until frozen 

In [2]:
# Imports
import sys
sys.path.append('/nsls2/software/mx/daq/platemate/src')

from pathlib import Path
import pandas as pd
from sqlalchemy import create_engine, func
from sqlalchemy.orm import sessionmaker
import ipywidgets as widgets
from models import XtalPlate, XtalPlateType, LibraryPlate, PuckType
from ui import EchoTransferWidget, XtalPlateCreatorWidget, BatchTimeStampWidget, IngestHarvestingDataWidget, GenerateHarvestDataWidget, LSDCDataGeneratorWidget
from utils.create import add_xtal_wells_to_plate
from utils.read import get_or_create

LAB_DATA_ROOT = Path("/nsls2/data/amx/proposals/commissioning/pass-311989/labdata_test_20240518")
engine = create_engine(f"sqlite:///{LAB_DATA_ROOT}/database/test.db")
Session = sessionmaker(bind=engine)
session = Session()

### Create an instance of a Xtal plate by: 
 1. Choosing the plate type from the dropdown menu
 2. Upload the imaging csv obtained from the plate shifter

In [11]:
imaging_path = LAB_DATA_ROOT / Path("imaging")
xtal_plate_creator = XtalPlateCreatorWidget(session=session, data_directory=imaging_path)
xtal_plate_creator.ui

### Create a new echo transfer
1. Select a library plate
2. Select a Xtal plate
3. Using slider, choose the number of xtal wells to be used
4. Change output file name (Optional)
5. Click "Generate Echo Transfer"

### Regenerate an existing batch
1. Choose the batch number in text box
2. Change output file name (Optional)
3. Click "Generate Echo Transfer"

**Note:** Library plate, Xtal plate and Xtal well slider values are ignored when generating an existing batch

## Show a list of all library plates in the db


In [19]:
echo_path = LAB_DATA_ROOT / Path("echo_protocols")
echo_transfer = EchoTransferWidget(session=session, output_folder=echo_path)
echo_transfer.ui

### Add time stamp to a batch

In [15]:
batch_timestamp = BatchTimeStampWidget(session=session)
batch_timestamp.ui

### Generate harvest file from batch


In [18]:
harvest_path = LAB_DATA_ROOT / Path("harvesting")
generate_harvest_data = GenerateHarvestDataWidget(session=session, output_folder=harvest_path)
generate_harvest_data.ui

### Ingest harvest file

In [20]:
ingest_harvest_data = IngestHarvestingDataWidget(session, data_directory=harvest_path)
ingest_harvest_data.ui

In [21]:
session.commit()

### LSDC puck data generator


In [22]:
puck_path = LAB_DATA_ROOT / Path("pucks")
lsdc_data_generator = LSDCDataGeneratorWidget(session=session, output_folder=puck_path)
lsdc_data_generator.ui